# Промежуточная аттестация. Применение Вопрос-ответного поиска

Для того чтобы реализовать вопрос-ответную систему, необходимо иметь базу данных, где хранится документация, информация содержащая ответ и тд. Среди нее мы будем искать информацию схожую по какой-либо метрике с поставленным вопросом.

Для этого задания потребуется применить какой-нибудь алгоритм для быстрого поиска нужных документов, содержащих что-либо связанной с вопросом. Для этого есть различные варианты, которые уже давно реализованы и даже не связанные с ИИ. Один из них <a href="https://en.wikipedia.org/wiki/Okapi_BM25">Okapi BM25</a>. В кратце это мешок слов с метрикой IDF, в результате чего каждый семпл текста из бд будет иметь свой ранг схожести с входной строкой.

In [84]:
!pip install rank_bm25 -q

In [85]:
import numpy as np
import pandas as pd
from pathlib import Path, PurePath

import nltk
nltk.download(['stopwords', 'punkt'])

from nltk.corpus import stopwords
import re
import string
import torch

from rank_bm25 import BM25Okapi # Search engine

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Для решения задачи был найден большой датасет, связанный с различной информацией по Covid - статьи, топики на сайтах и прочее (найденный на Kaggle).

In [90]:
from google.colab import drive

drive.mount('/content/drive/')
PATH_TO_DATA = 'drive/MyDrive'        # изменить на свое расположение

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [91]:
metadata_path = f'{PATH_TO_DATA}/covid_metadata.csv'
metadata_df = pd.read_csv(metadata_path, low_memory=False)
metadata_df = metadata_df.dropna(subset=['abstract', 'title']).reset_index(drop=True)

Для удобства создадим класс реализующий движок поиска по всем текстам в датасете, использующий алгоритм Okapi BM25.

In [92]:
from rank_bm25 import BM25Okapi

english_stopwords = list(set(stopwords.words('english')))

class CovidSearchEngine:
    def __init__(self, corpus: pd.DataFrame):
        self.corpus = corpus
        self.columns = corpus.columns

        # Объединим введение статьи и содержание
        raw_search_str = self.corpus.abstract.fillna('') + ' ' + self.corpus.title.fillna('')

        self.index = raw_search_str.apply(self.preprocess).to_frame()
        self.index.columns = ['terms']
        self.index.index = self.corpus.index
        self.bm25 = BM25Okapi(self.index.terms.tolist())

    def preprocess(self, text: str) -> list[str]:
        # предобработка текста (удаление спец символов, стоп слов, артиклей)
        return self.tokenize(self.remove_special_character(text.lower()))

    def remove_special_character(self, text: str) -> str:
        # удаляем пунктуацию
        return text.translate(str.maketrans('', '', string.punctuation))

    def tokenize(self, text: str) -> list[str]:
        words = nltk.word_tokenize(text)
        return list(set([word for word in words
                         if len(word) > 1
                         and not word in english_stopwords
                         and not word.isnumeric()
                        ])
                   )


    def search(self, query: str, num=3) -> pd.DataFrame:
        """
        Метод поиска `num` наиболее подходящих корпусов.
        Параметр (опц.) `num` - количество возвращаемых корпусов
        """
        # получаем оценки схожести вопроса и топиков (наибольшее значение - ближе всего)
        search_terms = self.preprocess(query)
        doc_scores = self.bm25.get_scores(search_terms)

        # берем `num` первых подходящих индексов топиков
        ind = np.argsort(doc_scores)[::-1][:num]

        # извлекаем наиболее подходящие контекст
        results = self.corpus.iloc[ind][self.columns]
        results['score'] = doc_scores[ind]
        results = results[results.score > 0]
        return results.reset_index()

In [93]:
cse = CovidSearchEngine(metadata_df)

In [94]:
!pip install transformers -q

Как всегда в Hugging Face уже выложены предобученные модели, основанные на BERT'e для задачи QA (Question Answering). Выберем модель, которая была предобучена на модели <a href="https://rajpurkar.github.io/SQuAD-explorer/">SQuAD</a> (Stanford Question Answering Dataset).

In [95]:
import torch
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

BERT_SQUAD = 'bert-large-uncased-whole-word-masking-finetuned-squad'

model = BertForQuestionAnswering.from_pretrained(BERT_SQUAD)
tokenizer = BertTokenizer.from_pretrained(BERT_SQUAD)

model = model.to(torch_device)
model.eval()

print()


Обучать модель в данном случае необходимости нет, преобученная BERT-QA модель получает на вход склеенные `input_ids` вопроса и контекста (в тензоре разделяемые через токен `[SEP]`) и выдает два типа логитов: начальные и конечные. По сути их можно интерпретировать как вероятности того, что данный токен является начальным токеном в ответе (конечным токеном соответственно для второго типа). По сути мы получаем два индекса в контексте: начало и конец подстроки ответа, и далее просто конвертируя токены получаем ответ.

In [96]:
def answer_question(question, context):
    encoded_dict = tokenizer.encode_plus(
        question,
        context,
        add_special_tokens=True,
        max_length=256,
        pad_to_max_length=True,
        return_tensors='pt'
    )

    input_ids = encoded_dict['input_ids'].to(torch_device)
    token_type_ids = encoded_dict['token_type_ids'].to(torch_device)

    output = model(input_ids, token_type_ids=token_type_ids)

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    start_index = torch.argmax(output.start_logits)
    end_index = torch.argmax(output.end_logits)

    answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])
    answer = answer.replace('[CLS]', '').replace('[SEP]', ' ')
    return answer

Для демонстрации необходимо в модель передать вопрос, далее будет выбрано определенное количество корпусов где будет искаться ответ на вопрос (по алгоритмы Okapi BM25) и соответственно будут возвращены ответы соответствующие выбранным корпусам.

In [97]:
NUM_CONTEXT_FOR_EACH_QUESTION = 3


def get_all_context(query, num_results):
    # Находит для данного вопроса `num_results` наиболее подходящих корпусов

    papers_df = cse.search(query, num_results)
    return papers_df['abstract'].str.replace("Abstract", "").tolist()


def get_all_answers(question, all_contexts):
    # Получение ответов от всех корпусов для данного вопроса

    all_answers = []

    for context in all_contexts:
        all_answers.append(answer_question(question, context))
    return all_answers


def create_output_results(question,
                          all_contexts,
                          all_answers,
                          summary_answer='',
                          summary_context=''):
    # Функция формирующая словарь со всеми заданными вопросами и ответами с корпусами
    output = {}
    output['question'] = question
    results = []
    for c, a in zip(all_contexts, all_answers):

        span = {}
        span['context'] = c
        span['answer'] = a
        results.append(span)

    output['results'] = results

    return output


def get_results(question,
                summarize=False,
                num_results=NUM_CONTEXT_FOR_EACH_QUESTION,
                verbose=True):
    # Входная точка демонстрации

    all_contexts = get_all_context(question, num_results)

    all_answers = get_all_answers(question, all_contexts)

    return create_output_results(question,
                                 all_contexts,
                                 all_answers)

In [98]:
questions =  [
    "How long is the incubation period for the virus?",
    "Can the virus be transmitted asymptomatically or during the incubation period?",
    "How does heart disease affect patients?",
    "How does smoking affect patients?",
    "How does pregnancy affect patients?",
    "What is the fatality rate of 2019-nCoV?",
    "Can animals transmit 2019-nCoV?",
    "What drugs or therapies are being investigated?",
    "Are anti-inflammatory drugs recommended?",
    "What telemedicine and cybercare methods are most effective?",
    "How is artificial intelligence being used in real time health delivery?",
    "What adjunctive or supportive methods can help patients?",
    "What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV?",
    "What is the immune system response to 2019-nCoV?",
    "Can 2019-nCoV infect patients a second time?"
]


In [99]:
all_answers = [get_results(q) for q in questions]

In [100]:
for q in all_answers:
    print('-'*42)
    print(f'Вопрос: {q["question"]}')
    for i, a in enumerate(q['results'], 1):
        print(f'Ответ {i}: {a["answer"]}')
    print('-'*42)


------------------------------------------
Вопрос: How long is the incubation period for the virus?
Ответ 1: long
Ответ 2: long
Ответ 3: 26 days of gestation
------------------------------------------
------------------------------------------
Вопрос: Can the virus be transmitted asymptomatically or during the incubation period?
Ответ 1: asymptomatic
Ответ 2: asymptomatically
Ответ 3: incubation period
------------------------------------------
------------------------------------------
Вопрос: How does heart disease affect patients?
Ответ 1: more susceptible
Ответ 2: the impact of concomitant cardiovascular disease on severity of covid - 19 was also evaluated . methods : a cross - sectional study was designed on 150 consecutive patients with covid - 19 in the fever clinic of tongji hospital in wuhan from january to february in 2020 , including 126 mild cases and 24 cases in critical care . both univariate and multivariate logistic regression were used to analyze the correlation of pas

По результатам демонстрации видим как и удачные ответы так и неудачные (3 пустых ответа в последнем тесте). Для улучшения предиктов можно увеличить количество сканируемых корпусов либо дообучить модель на данных по вопросам и ответам по Ковид

Весь этот функционал также доступен в библиотеке <a href="https://github.com/amaiya/ktrain/tree/master">`ktrain`</a> - обертка над TensorFlow, где на верхнем уровне и с удобным интерфейсом реализованы множество алгоритмо DeepLearning. Посмотрим как справится эта библиотека с той же задачей.

In [101]:
!pip install ktrain -q

In [102]:
metadata_df["raw_search_text"] = metadata_df.abstract.fillna('') + ' ' + metadata_df.title.fillna('')

In [103]:
import shutil
from ktrain.text import SimpleQA


INDEXDIR = '/tmp/myindex'
try:
    shutil.rmtree(INDEXDIR)
except FileNotFoundError:
    print('OK')
except OSError as e:
    print(e)
except Exception as e:
    raise e

In [104]:
docs = metadata_df["raw_search_text"].tolist()

In [105]:
# аналогично классу CovidSearchEngine в данной ячейке создается движок и индексы для быстрого поиска нужных контекстов

SimpleQA.initialize_index(INDEXDIR)
SimpleQA.index_from_list(
    docs,
    INDEXDIR,
    commit_every=len(docs),
    multisegment=True,
    procs=4,
    breakup_docs=True
)

In [106]:
# модель создается одной строчкой

qa = SimpleQA(INDEXDIR)

In [107]:
# всё, наконец-то демонстрация!)

answers = qa.ask('How long is the incubation period for the virus?')
qa.display_answers(answers[:3])

,Candidate Answer,Context,Confidence,Document Reference
0,is the delay from infection until onset of symptoms,"the incubation period is the delay from infection until onset of symptoms , and varies from person to person.",0.782764,16601
1,: 21 days,background : 21 days has been regarded as the appropriate quarantine period for holding individuals potentially exposed to ebola virus (ev) to reduce risk of contagion.,0.141524,15194
2,a short,the wells – riley equation for modelling airborne infection in indoor environments is incorporated into an seir epidemic model with a short incubation period to simulate the transmission dynamics of airborne infectious diseases in ventilated rooms.,0.046417,30864


Вопроса про инкубационный период: коротко и понятно 😆

In [74]:
answers = qa.ask('What is the fatality rate of 2019-nCoV?')
qa.display_answers(answers[:3])

,Candidate Answer,Context,Confidence,Document Reference
0,therefore 0. 3 % to 0. 6 %,"the infection fatality risk (ifr) — the actual risk of death among all infected individuals — is therefore 0. 3 % to 0. 6 % , which may be comparable to asian influenza pandemic of 1957 – 1958.",0.849170,38107
1,is 2. 2 %,"according to the released news, the case rate fatality is 2. 2 % (170 / 7824).",0.077169,35425
2,from 2 to 3 %,the case fatality rate is estimated to range from 2 to 3 % .,0.059859,37640


Вопрос про смертельные риски: Тут вроде все ок!

In [83]:
answers = qa.ask('Can 2019-nCoV infect patients a second time?')
qa.display_answers(answers[:3])

Увы и этот движок не смог найти ответы, что намекает на дообучение, изменение архитектуры модели или алгоритма.